# 4장 기본 신경망 구현

## 4.1 인공신경망의 작동 원리

** 인공신경망** 의 개념은 뇌를 구성하는 신경 세포, 즉 **뉴런**의 동작 원리에 기초한다.

뉴런의 기본 동작은 가지돌기에서 신호를 받아들이고, 이 신호가 축삭돌기를 지나 축삭단말로 전달되는 것이다. 다만 축삭돌기를 지나는 동안 신호가 약해지거나, 너무 약해서 축삭단말까지 전달되지 않거나, 또는 강하게 전달되기도 한다. 

그리고 축삭단말까지 전달된 신호는 연결된 다음 뉴런의 가지돌기로 전달된다. 우리는 이러한 원리를 지닌 수억 개의 뉴런 조합을 통해 손가락을 움직이거나 물체를 판별하는 등 다양한 조작과 판단을 수행한다.

**활성화 함수**(activation function)은 인공신경망을 통해 평가한 값을 최종적으로 어떤 값으로 만들지를 결정한다. 활성화 함수에는 대표적으로 **Sigmoid, ReLU, tanh** 함수가 있다.

**역전파(backpropagation)**는 출력층이 내놓은 결과의 오차를 신경망을 따라 입력층까지 역으로 전파하며 계산해나가는 방식이다. 이 방식은 입력층부터 가중치를 조절해가는 기존 방식보다 훨씬 유의미한 방식으로 가중치를 조절해 주어서 최적화 과정이 훨씬 빠르고 정확해진다.

## 4.2 간단한 분류 모델 구현하기

1. 털과 날개가 있느냐를 기준으로 포유류와 조류를 구분하는 신경망 모델을 만들어 보자.

In [1]:
import tensorflow as tf
import numpy as np

2. 학습에 사용할 데이터를 정의하자. 털과 날개가 있느냐를 담은 특징 데이터를 구성한다.

In [2]:
# [털, 날개]
x_data = np.array([[0,0 ], [1, 0], [1, 1], [0, 0], [0, 0], [0, 0]])

In [3]:
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

```py
# [털, 날개] -> [기타, 포유류, 조류]
[0, 0] - > [1, 0, 0] # 기타
[1, 0] - > [0, 1, 0] # 포유류
[1, 1] - > [0, 0, 1] # 조류
[0, 0] - > [1, 0, 0] # 기타
[0, 0] - > [1, 0, 0] # 기타
[0, 1] - > [0, 0, 1] # 조류

```

3. 신경망 모델을 구성해 보자. 특징 X와 레이블 Y의 관계를 알아내는 모델이다. 이때 X와 Y에 실측값(good truth)을 넣어서 학습시킬 것이니 X와 Y는 플레이스 홀더로 설정한다.

In [4]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

신경망을 결정하는 가중치와 편향 변수를 설정한다. 신경망은 이 변수들의 값을 여러 가지로 바꿔가면서 X와 Y의 연관 관계를 학습하게 된다.

In [5]:
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print (sess.run(W))
sess.close()

[[ 0.8721123   0.6598377  -0.42570376]
 [-0.35600114 -0.03803563  0.81663084]]


In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print (sess.run(b))
sess.close()

[0. 0. 0.]


가중치 변수 W는 [입력층(특징 수), 출력층(레이블 수)]의 구성인 [2, 3]으로 설정하고 평향 변수 b는 레이블 수인 3개의 요소를 가진 변수로 설정한다. 가중치를 곱하고 편향을 더한 결과를 활성화 함수인 ReLU에 적용하면 신경망 구성은 끝이다.

In [8]:
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

4. 신경망을 통ㅎ애 나온 출력값을 softmax 함수를 이용하여 사용하기 쉽게 다듬어준다.

In [9]:
model = tf.nn.softmax(L)

softmax 함수는 다음처럼 배열 내의 결과값을 전체 합이 1이 되도록 만들어준다. 전체가 1이니 각각은 해당 결과의 확률로 해석할 수 있다.

```py
[8.04, 2.76, -6.52] -> [0.53 0.24 0.23]
```

5. 손실함수를 작성해 보자. 손실 함수는 원-핫 인코딩을 이용한 대부분의 모델에서 사용하는 교차 엔트로피 함수를 사용하도록 하겠다. 교차 엔트로피 값은 예측값과 실제값 사이의 확률 분포 차이를 계산한 값으로, 기본 코드는 다음과 같다.

In [10]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

**Note** 손실 함수는 **비용 함수** 라고도 하며, 코드에서의 cost는 여기에서 온 것이다.

**Note** reduce_xxx 함수들은 테넛의 차원을 줄여준다. 함수 이름의 xxx 부분이 구체적인 차원 축소 방법을 뜻하고, axis로 축소할 차원을 정한다.

6. 학습을 시켜보자. 학습은 앞 장에서 배운 것처럼, 텐서플로가 기본 제공하는 최적화 함수를 제공하면 된다.

In [11]:
# 기본적인 경사하강법으로 최적화한다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화한다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행한다.
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습 도중 10번에 한 번씩 손실값을 출력한다.
    if (step + 1) % 10 == 0:
        print (step+1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 1.185543
20 1.175335
30 1.1655738
40 1.1562427
50 1.1473252
60 1.1388046
70 1.1313123
80 1.1246449
90 1.1182665
100 1.1121613


7. 마지막으로, 학습도니 결과를 확인해보는 코드를 작성하자

In [12]:
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print ('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print ('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [2 0 0 2 2 2]
실제값: [0 1 2 0 0 2]


8. 이제 정말 마지막으로 간단하게 정확도를 출력해 보자

In [13]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도: %2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도: 16.666668


전체 학습 데이터에 대한 예측값과 실측값을 tf.equal 함수로 비교한 뒤, true/false 값으로 나온 결과를 다시 tf.cast 함수로 이용해 0과 1로 바꾸어 평균을 내면 간단히 정확도를 구할 수 있다.

손실값이 점점 줄어들지만 학습 횟수를 아무리 늘려도 정확도가 크게 높아지지 않을 것이다. 그 이유는 신경망이 한 층밖에 안 되기 때문인데, 이는 층 하나만 더 늘리면 쉽게 해결된다.

### 전체 코드

In [34]:
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array([[0,0 ], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])


X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

model = tf.nn.softmax(L)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step + 1) % 10 == 0:
        print (step +1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print ('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print ('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도: %.2f' % sess.run(accuracy*100, feed_dict={X: x_data, Y: y_data}))

10 1.1897286
20 1.1832285
30 1.1769274
40 1.1707897
50 1.1649042
60 1.1591393
70 1.1535562
80 1.1481979
90 1.1429617
100 1.1383729
예측값: [0 2 2 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 66.67


## 4.3 심층 신경망 구현하기

신경망의 층을 둘 이상으로 구성한 심층 신경망. 즉, 딥러닝을 구현해 보자

1. 다층 신경망을 만드는 것은 매우 간단하다. 앞서 만든 신경망 모델에 가중치와 편향을 추가하기만 하면 된다.

In [15]:
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

코드를 보면 첫 번째 가중치의 형태는 [2, 10]으로, 두번째 가중치는 [10, 3]으로 설정하고 편향은 각각 10과 3으로 설정했다.

```py
# 가중치
W1 = [2, 10] -> [특징, 은닉층의 뉴런 수]
W2 = [10, 3] -> [은닉층의 뉴런 수, 분류 수]

# 편향
b1 = [10] -> 은닉층의 뉴런 수
b2 = [3] -> 분류 수
```

- 입려층과 출력층은 각각 특징과 분류 개수로 맞추고, 중간의 연결 부분은 맞닿은 층의 뉴런 수와 같도록 맞추면 된다. 중간의 연결 부분을 **은닉층** 이라 하며, 은닉 층의 뉴런수는 하이퍼파라미터나 실험을 통해 가장 적절한 수를 정하면 된다.

2. 특징 입력값에 첫 번째 가중치와 편향, 그리고 활성화 함수를 적용하자.

In [16]:
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

3. 다음으로 출력층을 만들기 위해 두 번째 가중치와 편향을 적용하여 최종 모델을 만든다. 은닉층에 두 번째 가중치 W2[10, 3]와 편향 b2[3]을 적용하면 최종적으로 3개의 출력값을 가지게 된다.

In [17]:
model = tf.add(tf.matmul(L1, W2), b2)

** NOTE ** 

- 앞 절에서 본 신경망 모델에서는 출력층에 활성화 함수를 적용하였으나, 사실 출력층에서는 보통 활성화 함수를 사용하지 않는다.
- 하이퍼 파라미터와 마찬가지로 은닉층과 출력층에서 활성화 함수를 사용할지 말지, 또 어떤 활성화 함수를 적용할지 정하는 일 또한 신경망 모델을 만드는데 가장 중요한 경험적, 실험적 요소이기도 하다.

4. 마지막으로 손실 함수를 작성한다. 손실 함수는 다시 한번 교차 엔트로피를 사용하되, 이번에는 텐서플로가 제공하는 교차 함수를 이용해 보자.

In [25]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

이번에는 최적화 함수로 AdamOptimizer를 사용해 보았다. 사용하는 최적화 함수에 따라 정확도나 학습 속도가 많이 달라질 수 있으며, AdamOptimizer는 앞서 사용한 GradientDescendentOptimizer보다 보편적으로 성능이 좋다고 알려져 있다. 하지만 모든 경우에 좋은 것은 아니다.

5. 학습 진행, 손실값과 정확도 측정 등, 앞 절에서 본 나머지 코드를 넣고 실행하면 다음과 같이 정확한 예측값을 얻게 될 것이다.

In [36]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step + 1) % 10 == 0:
        print (step +1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print ('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print ('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도: %.2f' % sess.run(accuracy*100, feed_dict={X: x_data, Y: y_data}))

10 0.91228867
20 0.75297
30 0.6189682
40 0.5025719
50 0.40509534
60 0.32203993
70 0.24936883
80 0.18500836
90 0.13273478
100 0.09496061
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


## 전체 코드

In [37]:
import tensorflow as tf
import numpy as np

x_data = np.array(
[[0, 0], [1, 0], [1,1],[0,0],[0,0],[0,1]])

y_data = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1],
])

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

model = tf.add(tf.matmul(L1, W2), b2)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step + 1) % 10 == 0:
        print (step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

prediction = tf.argmax(model, axis=1)
target = tf.arg_max(Y, 1)
print ('예측값:', sess.run(prediction, feed_dict={X:x_data}))
print ('실제값:', sess.run(target, feed_dict={Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

10 0.82943577
20 0.6863573
30 0.55944556
40 0.45815527
50 0.3742424
60 0.3088008
70 0.25737193
80 0.21749981
90 0.18560386
100 0.16041097
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
